In [ ]:
import pandas as pd
import os
# import seaborn as sns
import numpy as np
import re
from datetime import datetime
import seaborn as sns
path = './' + 'data/'
os.listdir(path)
# os.cpu_count()

['OD集货量',
 '中转场历史到件量0601至0831.csv',
 '场地数据.csv',
 '班次数据0601至0831.csv',
 '运力配载数据0601至0831']

In [170]:
# # %%time
# # trans = pd.read_json('./data/运力配载数据0601至0831/line_info_0601.json', lines=True)
# hist_arr = pd.read_csv(path + '中转场历史到件量0601至0831.csv')
loc = pd.read_csv(path + '场地数据.csv')
# # # loc.to_csv(path + '场地数据.csv', index = None, encoding='utf_8_sig')
# # task_arrange = pd.read_csv(path + '班次数据0601至0831.csv')


# path_od = path + 'OD集货量/'  
# # path+
# dfs = pd.DataFrame()
# for fname in os.listdir(path_od):
#     if re.search(r'\.csv$', fname):
#         dfs = pd.concat(
#             [dfs , pd.read_csv(path_od + fname)], axis = 0, ignore_index = True)
# prc_in = pd.DataFrame(dfs) 
# print(
#     'loc:', loc.shape, '\nhist:' ,
#     hist_arr.shape, '\ntask:',
#     task_arrange.shape, '\nod:' ,
#     dfs.shape)

In [61]:
task_loc = pd.read_pickle('./data_clean/task_loc.p')

---

# Loc table clean  

In [167]:
loc = pd.read_pickle('./data_clean/loc.p'); loc.shape

(444, 21)

In [168]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
loc[loc['场地代码'] == '552W']

,场地代码,场地类型,主服务业务区,单小时处理能力,卸货卡位总数,卸车卡位（17.5M）,卸车卡位（>=15M）,卸车卡位（<15M）,装货卡位总数,装车卡位（17.5M）,装车卡位（15M）,装车卡位（13.5M）,装车卡位（9.6M）,装车卡位（7.6M）,装车卡位（5.2M）,off,onn,off_p1,off_p2,onn_p1,onn_p2
205,552W,片区中转场,551Y,21516,12,False,True,True,87,False,True,True,False,False,False,22.77636,28.5,1.89803,0.261797,2.375,0.327586


In [171]:
# loc.columns
loc = loc.drop(['所在城市代码','应用场景', '分拨区' , '所在城市名称', '场地名称', '主服务城市','所在区部'], axis = 1) 
loc[[ '卸车卡位（17.5M）', '卸车卡位（>=15M）', '卸车卡位（<15M）', '装车卡位（17.5M）', '装车卡位（15M）',
       '装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）']]  = loc[[ '卸车卡位（17.5M）', '卸车卡位（>=15M）', '卸车卡位（<15M）', '装车卡位（17.5M）', '装车卡位（15M）',
       '装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）']] == 0
# loc[['装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）']].describe()
# loc.head(2)
loc['off_1'] = loc['卸车卡位（17.5M）'] * 17.5
loc['off_2'] = loc['卸车卡位（>=15M）'] * 15
loc['off_3'] = loc['卸车卡位（<15M）'] * (13.5 * 0.970721 + 9.6 * 11.056306 + 7.6 * 2.279279 + 5.2 * 9.826577)/ \
    (0.970721 + 11.056306 + 2.279279 + 9.826577)
loc['onn_1'] = loc['装车卡位（17.5M）'] * 17.5
loc['onn_2'] = loc['装车卡位（15M）'] * 15
loc['onn_3'] = loc['装车卡位（13.5M）'] * 13.5
loc['onn_4'] = loc['装车卡位（9.6M）'] * 9.6
loc['onn_5'] = loc['装车卡位（7.6M）'] * 7.6
loc['onn_6'] = loc['装车卡位（5.2M）'] * 5.2
loc['off'] = loc['off_1'] + loc['off_2'] + loc['off_3']
loc['onn'] = loc['onn_1'] + loc['onn_2'] + loc['onn_3'] + loc['onn_4'] + loc['onn_5'] + loc['onn_6']
loc = loc.drop(['off_1','off_2', 'off_3', 'onn_1', 'onn_2', 'onn_3', 'onn_4', 'onn_5', 'onn_6'], axis = 1)

loc['off_p1'] = loc['off']/(loc['卸货卡位总数']+1)
loc['off_p2'] = loc['off']/(loc['装货卡位总数']+1)
loc['onn_p1'] = loc['onn']/(loc['卸货卡位总数']+1)
loc['onn_p2'] = loc['onn']/(loc['装货卡位总数']+1)




In [ ]:
# loc.to_pickle('./data_clean/loc.p')
# corr_loc = loc.corr()
# corr_loc.style.background_gradient(cmap='coolwarm', axis=None)

,单小时处理能力,卸货卡位总数,卸车卡位（17.5M）,卸车卡位（>=15M）,卸车卡位（<15M）,装货卡位总数,装车卡位（17.5M）,装车卡位（15M）,装车卡位（13.5M）,装车卡位（9.6M）,装车卡位（7.6M）,装车卡位（5.2M）,off,onn,off_p1,off_p2,onn_p1,onn_p2
单小时处理能力,1.000000,0.694292,-0.213250,-0.248743,0.140098,0.515413,-0.314467,-0.201609,-0.124173,-0.018013,-0.122168,-0.107542,-0.340977,-0.400161,-0.507688,-0.442387,-0.514673,-0.435044
卸货卡位总数,0.694292,1.000000,-0.281807,-0.236958,0.148402,0.561361,-0.405929,-0.138989,-0.105247,-0.002723,-0.060192,-0.131470,-0.384069,-0.394709,-0.660113,-0.531848,-0.678136,-0.526231
卸车卡位（17.5M）,-0.213250,-0.281807,1.000000,-0.180776,-0.234207,-0.228785,0.508240,-0.129468,0.180945,-0.230001,-0.078532,0.098776,0.567338,0.200912,0.295329,0.207182,0.213142,0.164890
卸车卡位（>=15M）,-0.248743,-0.236958,-0.180776,1.000000,-0.350540,-0.139530,0.025541,0.385759,0.116778,0.017047,0.073878,0.027772,0.596186,0.305287,0.446809,0.339381,0.323826,0.250848
卸车卡位（<15M）,0.140098,0.148402,-0.234207,-0.350540,1.000000,0.188829,-0.155749,-0.114385,0.067248,0.368717,0.185574,0.116768,-0.047610,0.057440,-0.239251,-0.174016,-0.183933,-0.133413
装货卡位总数,0.515413,0.561361,-0.228785,-0.139530,0.188829,1.000000,-0.327459,-0.111249,-0.184515,0.037481,-0.049992,-0.440404,-0.231441,-0.395207,-0.501821,-0.519369,-0.534270,-0.529422
装车卡位（17.5M）,-0.314467,-0.405929,0.508240,0.025541,-0.155749,-0.327459,1.000000,-0.227340,0.244645,-0.155300,-0.026065,0.052813,0.378318,0.487021,0.332709,0.259858,0.342621,0.249270
装车卡位（15M）,-0.201609,-0.138989,-0.129468,0.385759,-0.114385,-0.111249,-0.227340,1.000000,0.024345,-0.149801,0.117706,0.053725,0.189454,0.443154,0.224204,0.178687,0.273056,0.195546
装车卡位（13.5M）,-0.124173,-0.105247,0.180945,0.116778,0.067248,-0.184515,0.244645,0.024345,1.000000,0.078580,0.166986,0.233030,0.290666,0.578570,0.165296,0.181117,0.207066,0.190376
装车卡位（9.6M）,-0.018013,-0.002723,-0.230001,0.017047,0.368717,0.037481,-0.155300,-0.149801,0.078580,1.000000,0.253774,-0.046161,-0.004974,0.243038,0.052770,0.093015,0.116395,0.131940


# Task arrage clean

In [ ]:
# task_arrange[(task_arrange['operate_zone_code'] == '552W') & (task_arrange['batch_date'] == '2021-06-04')]

In [65]:
# pd.to_pickle(ta,  './data_clean/task.p')
ta = pd.read_pickle('./data_clean/task.p')
# task_loc[task_loc['operate_zone_code'] == '010R']

In [ ]:
%%time

ta = task_arrange.groupby(['operate_zone_code', 'batch_date']).\
    agg(min_begin = ('plan_begin_tm', min),
    max_begin = ('plan_begin_tm', max),
    min_end = ('plan_end_tm', min),
    max_end = ('plan_end_tm', max),
    count = ('batch_code', 'nunique')
    
    ).reset_index()

def convt_min(col):
    ta[col] = ta[col].apply(lambda i: datetime.strptime(i,'%H:%M').hour*60 + datetime.strptime(i,'%H:%M').minute)
     
for i in ['min_begin', 'max_begin', 'min_end','max_end']:
    convt_min(i)


# ta['time_consume_per_vote'] = (ta['max_end'] - ta['min_begin'])/ta['count']

In [67]:
corr_ta = ta.corr()
corr_ta.style.background_gradient(cmap='coolwarm', axis=None)


,min_begin,max_begin,min_end,max_end,count,time_consume_per_vote
min_begin,1.000000,0.082276,0.370383,-0.547887,-0.614004,-0.711813
max_begin,0.082276,1.000000,-0.547942,0.151572,0.506862,-0.416152
min_end,0.370383,-0.547942,1.000000,0.129046,-0.577872,0.298075
max_end,-0.547887,0.151572,0.129046,1.000000,0.564497,0.712461
count,-0.614004,0.506862,-0.577872,0.564497,1.000000,0.187662
time_consume_per_vote,-0.711813,-0.416152,0.298075,0.712461,0.187662,1.000000


In [31]:
# %%time
# import matplotlib.pyplot as plt
# ta_plt2 = pd.DataFrame(ta['operate_zone_code'].value_counts())
# ta_plt2 = pd.DataFrame(ta_plt2['operate_zone_code'].value_counts())
# ta_plt2['site'] = ta_plt2.index
# plt.pie(x = 'operate_zone_code' , data =ta_plt2)

In [68]:
# %%time
# task_loc = ta.merge(loc, left_on='operate_zone_code', right_on = '场地代码', how = 'left')

In [ ]:
%%time
del loc, ta 
# task_loc.to_pickle('./data_clean/task_loc.p')


In [51]:
# %%time
# task_loc_valued = task_loc[~task_loc['场地代码'].isna()]
# task_loc_nulled = task_loc[task_loc['场地代码'].isna()]

In [ ]:
# task_loc.isnull().sum()/len(task_loc)

---
# od

In [7]:
od = pd.read_pickle('./data_clean/od.p');od.head(5)
"""
只搞了10天的,   历史到件量的场地不是每个都需要考虑的吧 集货OD对应的场地应该只有400出头 场地数据基本能覆盖的
"""

,srcsitecode,destsitecode,date1,batch_cnt,route_cnt,votes,weights,air_votes,air_weights
0,010R,010R,2021-06-01,2,4,10.0,122.500000,2.0,4.166667
1,010R,010R,2021-06-02,2,3,4.0,32.500000,0.0,0.000000
2,010R,010R,2021-06-03,2,4,15.0,102.416667,2.0,2.000000
3,010R,010R,2021-06-04,2,5,27.0,193.175000,2.0,2.900000
4,010R,010R,2021-06-05,1,2,2.0,2.000000,0.0,0.000000


In [ ]:
[prc_in[(prc_in['date1'] == '2021-06-08') & (prc_in['destsitecode'] == '552W')].sum(),
prc_in[(prc_in['date1'] == '2021-06-04') & (prc_in['srcsitecode'] == '552W')].sum()]

In [ ]:
# prc_in.head()

od = prc_in.groupby(['srcsitecode','destsitecode', 'date1']).agg(
    batch_cnt = ('srcbatchcode', 'nunique'),
    route_cnt = ('route_code', 'nunique'),
    votes = ('votes', sum),
    weights = ('weight1', sum),
    air_votes =('air_votes', sum),
    air_weights = ('air_weight1', sum)
).reset_index()

od_dest = od.groupby(['destsitecode', 'date1']).agg(
    {'batch_cnt': sum, 
    'route_cnt': sum,
    'votes': sum, 
    'weights': sum,
    'air_votes': sum, 
    'air_weights':sum
    }).reset_index();od_dest

In [ ]:
# od.to_pickle('./data_clean/od.p')

In [ ]:
# hist_arr[hist_arr['a.report_dt'] == '2021-06-01'].sort_values('a.zonecode')

In [21]:
[od.date1.unique(), od.date1.max()]

[array(['2021-06-01', '2021-06-02', '2021-06-03', '2021-06-04',
        '2021-06-05', '2021-06-06', '2021-06-07', '2021-06-08',
        '2021-06-09', '2021-06-10'], dtype=object),
 '2021-06-10']

---

# hist

In [42]:
hist_arr[(hist_arr['a.zonecode'] == '552W') & ( hist_arr['a.report_dt'] == '2021-06-04' )]

,a.report_dt,a.zonecode,all_arr_waybill,all_arr_weight,simu_arr_waybill,simu_arr_weight
35110,2021-06-04,552W,175334,472758.782153,6500,13488.254667


# hist_arr.to_pickle('./data_clean/hist_arr.p')
# merge all

In [10]:
hist_arr = pd.read_pickle('./data_clean/hist_arr.p')
hist_arr.head()

,a.report_dt,a.zonecode,all_arr_waybill,all_arr_weight,simu_arr_waybill,simu_arr_weight
0,2021-06-08,010CC006,525,1280.400000,0,0.000000
1,2021-06-08,010CC007,0,0.000000,0,0.000000
2,2021-06-08,010R,102944,230415.243664,76818,165515.803556
3,2021-06-08,010RA,109154,249089.557278,109154,249089.557278
4,2021-06-08,010RG,119183,261849.663955,118678,260439.168955


In [24]:
hist_arr_part = hist_arr[hist_arr['a.report_dt'] < '2021-06-11']

--- 
# merge od_dest, ta, loc, hist_arr_part

In [75]:
[ta.columns,
od_dest.columns,
loc.columns,
hist_arr_part.columns]

[Index(['operate_zone_code', 'batch_date', 'min_begin', 'max_begin', 'min_end',
        'max_end', 'count', 'time_consume_per_vote'],
       dtype='object'),
 Index(['destsitecode', 'date1', 'batch_cnt', 'route_cnt', 'votes', 'weights',
        'air_votes', 'air_weights'],
       dtype='object'),
 Index(['场地代码', '场地类型', '主服务业务区', '单小时处理能力', '卸货卡位总数', '卸车卡位（17.5M）',
        '卸车卡位（>=15M）', '卸车卡位（<15M）', '装货卡位总数', '装车卡位（17.5M）', '装车卡位（15M）',
        '装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）', 'off', 'onn',
        'off_p1', 'off_p2', 'onn_p1', 'onn_p2'],
       dtype='object'),
 Index(['a.report_dt', 'a.zonecode', 'all_arr_waybill', 'all_arr_weight',
        'simu_arr_waybill', 'simu_arr_weight'],
       dtype='object')]

In [86]:
"""
od_dest , ta, loc, hist_arr_part
"""
od_ta = od_dest.merge(ta, \
    left_on=['destsitecode', 'date1'], 
    right_on = ['operate_zone_code', 'batch_date'],
    how = 'left',  validate = 'one_to_one') 
    # task arrange lakc 31

# od_dest.merge(loc, \
#     left_on=['destsitecode' ], 
#     right_on = ['场地代码'],
#     how = 'left')
    #  .isna().sum() 1388

od_ta_his = hist_arr_part.merge(
    od_ta, 
    left_on = ['a.report_dt', 'a.zonecode'], 
    right_on = ['date1', 'destsitecode'], validate = 'one_to_one', how = 'right') \
        # .isna().sum()   264

In [91]:
od_ta_his = od_ta_his.drop(['a.report_dt', 'a.zonecode','operate_zone_code','batch_date'], axis= 1)

In [143]:
impute = pd.DataFrame(od_ta_his.isna().sum() != 0)
impute = impute[impute[0] == True];impute

,0
all_arr_waybill,True
all_arr_weight,True
simu_arr_waybill,True
simu_arr_weight,True
min_begin,True
max_begin,True
min_end,True
max_end,True
count,True
time_consume_per_vote,True


In [ ]:
del loc

In [ ]:
# imp.fit_transform(od_ta_his['all_arr_weight'].array.reshape(-1,1))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [152]:
# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.impute import SimpleImputer, KNNImputer
imp = SimpleImputer('median')
imp.fit_transform(od_ta_his[list(impute.index)])

C:\Users\dscshap3808\Miniconda3\envs\siming\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass missing_values=median as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


NotFittedError: This SimpleImputer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [87]:
# hist_od_part = hist_arr_part.merge(
#     od_dest, 
#     left_on = ['a.report_dt', 'a.zonecode'], 
#     right_on = ['date1', 'destsitecode'], validate = 'one_to_one', how = 'right'); len(hist_od_part)

# hist_od_part = hist_od_part.drop(['a.report_dt','a.zonecode'], axis =1 )

In [44]:
hist_od_part

,all_arr_waybill,all_arr_weight,simu_arr_waybill,simu_arr_weight,destsitecode,date1,batch_cnt,route_cnt,votes,weights,air_votes,air_weights
0,117528.0,237472.306251,85223.0,165893.494172,010R,2021-06-01,9,11,16.0,140.800000,4.0,33.666667
1,114545.0,242443.357334,81537.0,168838.271100,010R,2021-06-02,7,8,8.0,34.600000,2.0,32.500000
2,97133.0,214623.247696,70183.0,152144.916293,010R,2021-06-03,8,10,17.0,106.216667,6.0,12.400000
3,102903.0,224132.253492,74137.0,155885.942530,010R,2021-06-04,4,7,28.0,194.175000,3.0,10.400000
4,64644.0,150821.001720,48298.0,106059.225291,010R,2021-06-05,5,6,7.0,15.333333,1.0,13.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
4898,8165.0,29186.366667,8165.0,29186.366667,SIN01R,2021-06-06,141,104,0.0,0.000000,285.0,1264.000000
4899,NaN,NaN,NaN,NaN,SIN01R,2021-06-07,265,167,0.0,0.000000,950.0,4844.884615
4900,139.0,794.960000,139.0,794.960000,SIN01R,2021-06-08,256,157,0.0,0.000000,923.0,3888.000000
4901,1614.0,3380.180000,1614.0,3380.180000,SIN01R,2021-06-09,242,150,0.0,0.000000,933.0,4632.535556


In [62]:
train = task_loc.merge(hist_od_part, \
    left_on = ['operate_zone_code', 'batch_date'], right_on=['destsitecode', 'date1'], 
    validate = 'one_to_one',
    how = 'right')

In [63]:
train.isna().sum()

operate_zone_code          31
batch_date                 31
min_begin                  31
max_begin                  31
min_end                    31
max_end                    31
count                      31
time_consume_per_vote      31
场地代码                     1388
场地类型                     1388
主服务业务区                   1388
单小时处理能力                  1388
卸货卡位总数                   1388
卸车卡位（17.5M）              1388
卸车卡位（>=15M）              1388
卸车卡位（<15M）               1388
装货卡位总数                   1388
装车卡位（17.5M）              1388
装车卡位（15M）                1388
装车卡位（13.5M）              1388
装车卡位（9.6M）               1388
装车卡位（7.6M）               1388
装车卡位（5.2M）               1388
off                      1388
onn                      1388
off_p1                   1388
off_p2                   1388
onn_p1                   1388
onn_p2                   1388
all_arr_waybill           264
all_arr_weight            264
simu_arr_waybill          264
simu_arr_weight           264
destsiteco

In [53]:
# del od, task_loc, hist_arr
# del task_loc_nulled, task_loc_valued
# # [len(task_loc), len(train), len(od)]

---
# transport


In [57]:
# trans.head(2)

In [ ]:
trans.iloc[11,:]['passZoneCodes'][0]

In [ ]:
trans.iloc[11,:]['taskInfos'][0] # ['unloadZone']

In [ ]:
trans.iloc[1,:]['vehicleInfos'][0] # ['unloadZone']

---
# what a model

In [ ]:
# train_full = train_full_sktime
train_full = train_full.asfreq('D')
train_full = train_full.fillna(0) 
train_first = train_first.asfreq('D')
train_first = train_first.fillna(0)
train_full['CREATION_DATE_TIME_STAMP'] = train_full.index
train_first['CREATION_DATE_TIME_STAMP'] = train_first.index


# train_full
train_full = train_full.reset_index(drop =True)
train_first = train_first.reset_index(drop =True)

train_fin = train_full.merge(train_first, 
    on = 'CREATION_DATE_TIME_STAMP', 
    how = 'outer',
    suffixes = ['_full', '_first'])
train_fin = train_fin.fillna(0);train_fin


# from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.model_selection import train_test_split

x = train_first 
x['y'] = train_fin['qty_full']
# x['CREATION_DATE_TIME_STAMP'] = x.index
x_train, x_test = train_test_split(x, train_size = .75, shuffle = False, random_state = 52943)

cols = [i for i in x.columns if i not in ['y', 'CREATION_DATE_TIME_STAMP']]


In [56]:
%%time

from fbprophet import Prophet
# df = x[['CREATION_DATE_TIME_STAMP', 'qty']]
df_naiv = x_train 
df_naiv = df_naiv.rename({'CREATION_DATE_TIME_STAMP' : 'ds'}, axis = 1)
m = Prophet()
[m.add_regressor(i) for i in cols]

m.fit(df_naiv)

from sklearn.metrics import mean_absolute_percentage_error
df_naiv_test = x_test
df_naiv_test = df_naiv_test.rename({'CREATION_DATE_TIME_STAMP' : 'ds'}, axis = 1)
# m.predict(x_test)
mean_absolute_percentage_error(df_naiv_test['y'], m.predict(df_naiv_test)['yhat'])
# fig = m.plot_components(m.predict(x_test))

'c:\\Users\\dscshap3808\\Documents\\my_scripts_new\\play7_comp'